In [1]:
import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.models.DirectRanker import DirectRanker
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [2]:
def generate_direct_ranker_dataset(num_samples, num_features):
    # Erzeugen von zwei Tensoren mit zufälligen Daten
    tensor_A = torch.randn(num_samples, num_features, dtype=torch.float32)
    tensor_B = torch.randn(num_samples, num_features, dtype=torch.float32)

    # Erzeugen von zufälligen Labels (0 oder 1)
    labels = torch.randint(0, 2, (num_samples,), dtype=torch.float32)

    return tensor_A, tensor_B, labels

# Beispiel: Erzeugen eines Datensatzes mit 100 Samples, jeder mit 10 Features
num_samples = 1000
num_features = 14
tensor_A, tensor_B, labels = generate_direct_ranker_dataset(num_samples, num_features)

print(tensor_A.shape, tensor_B.shape, labels.shape)


torch.Size([1000, 14]) torch.Size([1000, 14]) torch.Size([1000])


In [2]:
model = DirectRanker(num_features=14)
n_epochs = 500
#print(list(model.parameters()))
lr = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
# training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(tensor_A, tensor_B)
    y_pred = torch.squeeze(y_pred)
    loss = loss_fn(labels, y_pred)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:    
        print(loss)


NameError: name 'tensor_A' is not defined

In [6]:
print(model)

DirectRanker(
  (feature_activation): Tanh()
  (ranking_activation): Sigmoid()
  (layers): Sequential(
    (0 layer): Linear(in_features=14, out_features=10, bias=True)
    (0 activation): Tanh()
    (1 layer): Linear(in_features=10, out_features=5, bias=True)
    (1 activation): Tanh()
  )
  (ranking_layer): Linear(in_features=5, out_features=1, bias=False)
  (ranking_layer_cls): Linear(in_features=5, out_features=1, bias=False)
)


In [5]:
for name, param in model.named_parameters():
    print(name)

layers.0 layer.weight
layers.0 layer.bias
layers.1 layer.weight
layers.1 layer.bias
ranking_layer.weight
ranking_layer_cls.weight


In [27]:
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki

#data = Law('race')
#data = Adult()
#data = Compas()
data = Wiki()

FileNotFoundError: [Errno 2] No such file or directory: 'data//toxicity_annotated_comments.tsv'

In [20]:
from FairRanking.models.BaseDirectRanker import build_pairs
X = np.array(data.x_col)
y = np.array(data.y_col)
X0, X1, y = build_pairs(X, y, 1000)
X0 = torch.tensor(X0, dtype=torch.float32)
X1 = torch.tensor(X1, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [21]:
X0.shape, y.shape

(torch.Size([9000, 8]), torch.Size([9000, 1]))

In [26]:
# law dataset works
# adult have 0 gradients after a few epoch, regardless model complexity
# compas only learns very slow, regardless model complexity
model = DirectRanker(num_features=X0.shape[1], hidden_layers=[16]) # more hidden layers => gradient becomes zero and loss high
n_epochs = 1000
lr = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
# training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X0, X1)
    y_pred = torch.squeeze(y_pred)
    loss = loss_fn(y, y_pred)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:    
        print(loss)
        print(y_pred.dtype)
        for name, param in model.named_parameters():
            if param.grad is None:
                print(name)
            else:
                print(name, param.grad.data.sum())
                break


tensor(1.0468, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(0.3325)
tensor(0.9490, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(-0.0585)
tensor(0.9477, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(0.0004)
tensor(0.9474, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(-0.0102)
tensor(0.9472, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(-2.5849e-05)
tensor(0.9471, grad_fn=<MseLossBackward0>)
torch.float32
layers.0.weight tensor(0.0335)


KeyboardInterrupt: 

In [18]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

10108